- **EVENT_ON_MOUSE_MOVE &#8594; 0**
- **EVENT_ON_TOUCH_MOVE &#8594; 7**
- **EVENT_ON_CLICK &#8594; 1**
- **EVENT_ON_DOUBLE_CLICK &#8594; 2**
- **EVENT_ON_MOUSE_DOWN &#8594; 3**
- **EVENT_ON_MOUSE_UP &#8594; 4**
- **EVENT_ON_WHEEL &#8594; 5**
- **EVENT_CONTEXT_MENU &#8594; 6**
- **EVENT_WINDOW_SCROLL &#8594; 11**
- **EVENT_WINDOW_RESIZE &#8594; 12**
- **EVENT_KEY_DOWN &#8594; 13**
- **EVENT_KEY_PRESS &#8594; 14**
- **EVENT_KEY_UP &#8594; 15**
- **EVENT_FOCUS &#8594; 16**
- **EVENT_BLUR &#8594; 17**
- **EVENT_ON_CHANGE_SELECTION_OBJECT &#8594; 18**
- **EVENT_ON_CLICK_SELECTION_OBJECT &#8594; 19**
- **EVENT_INIT_TRACKING &#8594; 100**
- **EVENT_TRACKING_END &#8594; 200**
- **COMPONENT_COMBOBOX &#8594; 2**
- **COMPONENT_OPTION &#8594; 3**

In [1]:
# %pip install numpy pandas scikit-learn matplotlib seaborn --quiet
#
# import matplotlib.pyplot as plt
# import pandas as pd
# import numpy as np
#
# pd.set_option('display.max_columns', None)
# random_state = 42
#
# dataset_path1 = 'C:/Users/Usuario/TFM/all_sessions.csv'
#
# df1 = pd.read_csv(dataset_path1,
#                   usecols=['sceneId', 'eventType', 'timeStamp', 'x', 'y', 'sessionId'],
#                   dtype={
#                       'sceneId': 'str',
#                       'eventType': 'int',
#                       'timeStamp': 'int',
#                       'x': 'int',
#                       'y': 'int',
#                       'sessionId': 'str'
#                   })
#
# dataset_path2 = 'data/dataset_small.csv'
#
# df2 = pd.read_csv(dataset_path2,
#                   usecols=['sceneId', 'eventType', 'timeStamp', 'x', 'y', 'sessionId'],
#                   dtype={
#                       'sceneId': 'str',
#                       'eventType': 'int',
#                       'timeStamp': 'int',
#                       'x': 'int',
#                       'y': 'int',
#                       'sessionId': 'str'
#                   })
#
# df = pd.concat([df1, df2], ignore_index=True)
#
# #df.to_csv('data/dataset_all.csv', index=False)


In [2]:
# # Count the number of duplicates in the dataset
# df.duplicated().sum()
# # Remove duplicates
# df = df.drop_duplicates()

In [3]:
# df.columns

In [4]:
# def calcular_metricas(df):
#     df = df.copy()
#     df.sort_values(by=['timeStamp'], inplace=True)
#     df['timeStamp'] = pd.to_numeric(df['timeStamp'], errors='coerce')
#
#     df['dt'] = df.groupby(['sessionId', 'sceneId'])['timeStamp'].diff().fillna(0)
#     df['dx'] = df.groupby(['sessionId', 'sceneId'])['x'].diff().fillna(0)
#     df['dy'] = df.groupby(['sessionId', 'sceneId'])['y'].diff().fillna(0)
#
#     df['distance'] = np.sqrt(df['dx'] ** 2 + df['dy'] ** 2)
#     df['velocity'] = df['distance'] / df['dt']
#     df['acceleration'] = df.groupby(['sessionId', 'sceneId'])['velocity'].diff().fillna(0) / df['dt']
#
#     return df
#
#
# def filtrar_datos_validos(df):
#     df_valid = df[
#         (df['x'] > 0) &
#         (df['y'] > 0) &
#         (df['eventType'] == 7) &
#         (df['timeStamp'] > 0)
#         ].copy()
#     return df_valid.sort_values(['timeStamp'])
#
#
# def segmentar_trazos(df_valid, time_threshold=200):
#     strokes = []
#     if len(df_valid) < 2:
#         return strokes
#
#     current_stroke = [df_valid.index[0]]
#
#     for i in range(1, len(df_valid)):
#         current_idx = df_valid.index[i]
#         previous_idx = df_valid.index[i - 1]
#         time_diff = abs(df_valid.loc[current_idx, 'y'] - df_valid.loc[previous_idx, 'y'])
#
#         if time_diff <= time_threshold:
#             current_stroke.append(current_idx)
#         else:
#             if len(current_stroke) >= 2:
#                 strokes.append(current_stroke)
#             current_stroke = [current_idx]
#
#     if len(current_stroke) >= 2:
#         strokes.append(current_stroke)
#
#     return strokes
#
#
# def visualizar_trazo(df, stroke_indices, stroke_id):
#     stroke_data = df.loc[stroke_indices]
#     metrics = calcular_metricas(stroke_data)
#
#     plt.figure(figsize=(10, 8))
#     plt.plot(stroke_data['x'], stroke_data['y'], 'b-o', linewidth=2, markersize=4, label='Trazo real')
#
#     x_start, y_start = stroke_data['x'].iloc[0], stroke_data['y'].iloc[0]
#     x_end, y_end = stroke_data['x'].iloc[-1], stroke_data['y'].iloc[-1]
#     plt.plot([x_start, x_end], [y_start, y_end], 'r--', linewidth=2, label='Línea óptima')
#
#     plt.plot(x_start, y_start, 'go', markersize=8, label='Inicio')
#     plt.plot(x_end, y_end, 'ro', markersize=8, label='Fin')
#
#     duration = stroke_data['timeStamp'].iloc[-1] - stroke_data['timeStamp'].iloc[0]
#     plt.xlabel('X (píxeles)')
#     plt.ylabel('Y (píxeles)')
#     plt.title(f'Trazo {stroke_id} - Duración: {duration:.0f}ms - Puntos: {len(stroke_data)}')
#     plt.legend()
#     plt.grid(True, alpha=0.3)
#     plt.gca().invert_yaxis()
#     plt.show()
#
#     return metrics
#
#
# def procesar_trazo(df, stroke_id):
#     df_valid = filtrar_datos_validos(df)
#     strokes = segmentar_trazos(df_valid)
#
#     if not strokes:
#         print("No se encontraron trazos válidos")
#         return
#
#     if stroke_id >= len(strokes):
#         print(f"No se encontró el trazo {stroke_id}. Trazos disponibles: 0-{len(strokes) - 1}")
#         return
#
#     print(f"Total de trazos encontrados: {len(strokes)}")
#     for i, stroke in enumerate(strokes):
#         stroke_duration = df.loc[stroke[-1], 'timeStamp'] - df.loc[stroke[0], 'timeStamp']
#         print(f"Trazo {i}: {len(stroke)} puntos, duración: {stroke_duration:.0f}ms")
#
#     return visualizar_trazo(df, strokes[stroke_id], stroke_id)
#
#
# def calcular_metricas_trazos(df):
#     resultados = []
#
#     # Filtrar datos válidos
#     df_valid = df[
#         (df['x'] > 0) &
#         (df['y'] > 0) &
#         (df['eventType'] == 7) &
#         (df['timeStamp'] > 0)
#         ].copy()
#
#     df_valid = df_valid.sort_values(['sessionId', 'sceneId', 'timeStamp'])
#
#     # Agrupar por sessionId y sceneId
#     for (session, scene), grupo in df_valid.groupby(['sessionId', 'sceneId']):
#         grupo = grupo.reset_index(drop=True)
#         strokes = []
#         current_stroke = [0]
#         time_threshold = 200
#
#         for i in range(1, len(grupo)):
#             time_diff = abs(grupo.loc[i, 'timeStamp'] - grupo.loc[i - 1, 'timeStamp'])
#             if time_diff <= time_threshold:
#                 current_stroke.append(i)
#             else:
#                 if len(current_stroke) >= 2:
#                     strokes.append(current_stroke)
#                 current_stroke = [i]
#
#         if len(current_stroke) >= 2:
#             strokes.append(current_stroke)
#
#         for stroke_id, indices in enumerate(strokes):
#             trazo = grupo.loc[indices].copy()
#             trazo['dt'] = trazo['timeStamp'].diff().replace(0, np.nan)
#             trazo['dx'] = trazo['x'].diff()
#             trazo['dy'] = trazo['y'].diff()
#             trazo['distance'] = np.sqrt(trazo['dx'] ** 2 + trazo['dy'] ** 2)
#             trazo['velocity'] = trazo['distance'] / trazo['dt']
#             trazo['acceleration'] = trazo['velocity'].diff() / trazo['dt']
#
#             trazo = trazo.replace([np.inf, -np.inf], np.nan).dropna()
#
#             if len(trazo) < 2:
#                 continue
#
#             duration = trazo['timeStamp'].iloc[-1] - trazo['timeStamp'].iloc[0]
#             total_distance = trazo['distance'].sum()
#             avg_velocity = trazo['velocity'].mean()
#             max_velocity = trazo['velocity'].max()
#             min_velocity = trazo['velocity'].min()
#             std_velocity = trazo['velocity'].std()
#             avg_acceleration = trazo['acceleration'].mean()
#             max_acceleration = trazo['acceleration'].max()
#             min_acceleration = trazo['acceleration'].min()
#             std_acceleration = trazo['acceleration'].std()
#
#             # Área bajo la curva real
#             area_real = np.trapezoid(trazo['y'], trazo['x'])
#
#             # Área bajo la línea óptima
#             x0, y0 = trazo['x'].iloc[0], trazo['y'].iloc[0]
#             x1, y1 = trazo['x'].iloc[-1], trazo['y'].iloc[-1]
#             x_opt = np.linspace(x0, x1, len(trazo))
#             y_opt = np.linspace(y0, y1, len(trazo))
#             area_optima = np.trapezoid(y_opt, x_opt)
#
#             area_diff_ratio = abs(area_real - area_optima) / (abs(area_optima) + 1e-6)
#
#             resultados.append({
#                 'sessionId': session,
#                 'sceneId': scene,
#                 'stroke_id': stroke_id,
#                 'duration_ms': duration,
#                 'total_distance': total_distance,
#                 'avg_velocity': avg_velocity,
#                 'max_velocity': max_velocity,
#                 'min_velocity': min_velocity,
#                 'std_velocity': std_velocity,
#                 'avg_acceleration': avg_acceleration,
#                 'max_acceleration': max_acceleration,
#                 'min_acceleration': min_acceleration,
#                 'std_acceleration': std_acceleration,
#                 'num_points': len(trazo),
#                 'area_real': area_real,
#                 'area_optima': area_optima,
#                 'area_diff_ratio': area_diff_ratio
#             })
#
#     return pd.DataFrame(resultados)

In [5]:
# metricas_trazos = calcular_metricas_trazos(df)

In [6]:
# def calcular_metricas_globales_por_sesion_escena(df_metricas_trazos):
#     """
#     Calcula métricas globales agregadas para cada combinación de sessionId y sceneId.
#
#     Parámetros:
#     - df_metricas_trazos: DataFrame con métricas por trazo, incluyendo columnas como:
#         ['sessionId', 'sceneId', 'duration_ms', 'total_distance', 'avg_velocity', 'max_velocity',
#          'avg_acceleration', 'max_acceleration', 'area_real', 'area_optima', 'area_diff_ratio']
#
#     Retorna:
#     - DataFrame con métricas globales agregadas por (sessionId, sceneId)
#     """
#     # Definir las métricas a agregar y sus funciones de agregación
#     metricas = {
#         'duration_ms': ['mean', 'max', 'min', 'std'],
#         'total_distance': ['mean', 'max', 'min', 'std'],
#         'avg_velocity': ['mean', 'max', 'min', 'std'],
#         'max_velocity': ['mean', 'max', 'min', 'std'],
#         'avg_acceleration': ['mean', 'max', 'min', 'std'],
#         'max_acceleration': ['mean', 'max', 'min', 'std'],
#         'area_real': ['mean', 'max', 'min', 'std'],
#         'area_optima': ['mean', 'max', 'min', 'std'],
#         'area_diff_ratio': ['mean', 'max', 'min', 'std'],
#         'num_points': ['sum', 'mean']
#     }
#
#     # Agrupar por sessionId y sceneId y aplicar agregaciones
#     df_global = df_metricas_trazos.groupby(['sessionId', 'sceneId']).agg(metricas)
#
#     # Aplanar columnas jerárquicas
#     df_global.columns = ['_'.join(col).strip() for col in df_global.columns.values]
#     df_global = df_global.reset_index()
#
#     return df_global

In [7]:
# df_global = calcular_metricas_globales_por_sesion_escena(metricas_trazos)

In [8]:
#df_global.to_csv('data/dataset_all_global.csv', index=False)

In [9]:
# procesar_trazo(df[(df['sceneId']=='LA MANO_SYMBOLIC_I-I.phase1') & (df['sessionId']=='AAZ8612490310781749024507588642025')], 0)

In [10]:
# import pandas as pd
#
# dataset_path = 'data/dataset_all_global.csv'
# df = pd.read_csv(dataset_path)
#
# with open('unique_session_ids.txt', 'w') as f:
#     for session_id in df['sessionId'].unique():
#         f.write(f"{session_id}\n")

In [11]:
# ids = [
#     "ADS8787368630791749452549957692025",
#     "ATB9200223209131749452177652692025",
#     "AVY6543686901691748849082100622025",
#     "BET96942045227117496262637716112025",
#     "BLO3577167625591748849252825622025",
#     "CIY2243931294961749455766044692025",
#     "DDT30123260255317485927900265302025",
#     "DFS74366367530517496262176546112025",
#     "DJS93025746822017485924646175302025",
#     "DLQ44195863626517484171906615282025",
#     "DOE64291218906917496285834516112025",
#     "DSM6484826647291748849214177622025",
#     "DUY26361033726917478145048385212025",
#     "DYI39831232704317478141202555212025",
#     "EHO5349775278391749023943141642025",
#     "FAY4438337027851749023584621642025",
#     "FDV8826417879691749196020367662025",
#     "FJF9140438477411749452478356692025",
#     "GXP7903476890701749194891680662025",
#     "HII9783628243151749452142340692025",
#     "HOK2350810650281749455783722692025",
#     "HXE3427797618191749022282982642025",
#     "ILE78972162668117485923370255302025",
#     "JET69975656969117496252265446112025",
#     "JIN70495937325217484203217395282025",
#     "JKX2543026792341749023470388642025",
#     "JVO58554806084517496264335716112025",
#     "KAA19486048295117478140745725212025",
#     "KPM2031804782861749453815036692025",
#     "LBH25850875310017476376301045192025",
#     "LHE47072582766017478148073885212025",
#     "LKP19562889629917485913003495302025",
#     "LLF3262063649481748850541938622025",
#     "LNT6645552579441749456281025692025",
#     "LWH35924425239217485926290575302025",
#     "MUG8937952666901748849206294622025",
#     "MYA34748800846817485923333625302025",
#     "NKB64129577257117478146607205212025",
#     "PBM6053058426031749025169416642025",
#     "PCU7102056409081749454889234692025",
#     "PHJ9187348855881749021884688642025",
#     "PIN2895781986571749023092838642025",
#     "PKP4502355851251749195522742662025",
#     "QEG44438359903017484187201745282025",
#     "QLE77121522033217496261478006112025",
#     "QSU4508784962681749455815282692025",
#     "RWC3651180137111749024748699642025",
#     "RWG4460878523761749025785704642025",
#     "TIX83864307729517484183796395282025",
#     "UDL3367809292011749196951901662025",
#     "WLC78734424558217484206179215282025",
#     "WOG70047012833617496278837966112025",
#     "WVZ7801004924481749021759735642025",
#     "WZY71365918801617476424184875192025",
#     "XAU2751312518661748849489133622025",
#     "XLX1142928749471749022123984642025",
#     "YDN7338036580751749024780093642025",
#     "YKO48872277741817496261167556112025",
#     "ZAH42060067917217476422144425192025",
#     "ZND86434197891017484183540685282025",
#     "ZPB3797781205041749197116548662025",
#     "ZVL53621028266817485922319715302025",
# ]
#
# df_filtered = df[~df['sessionId'].isin(ids)].copy()
#
# session_to_user = {"AAZ8612490310781749024507588642025": "171",
#                    "ABT67785723459417496249541446112025": "221",
#                    "ACD5112291518021749454523207692025": "183",
#                    "ACX4523446943281748849024560622025": "175",
#                    "ACZ4875654388041749193170556662025": "166",
#                    "AEI28612769026617496271145906112025": "153",
#                    "AES6275308060691749022823158642025": "197",
#                    "AFK74450571383617484194659635282025": "198",
#                    "AGP46037880266917476412054125192025": "178",
#                    "AHT4783292001631748850091356622025": "183",
#                    "AIT10679636444917478111666735212025": "215",
#                    "AJR57335404259117478135473665212025": "158",
#                    "AMG49139198765917485920786015302025": "169",
#                    "ANC2113295573921749193115728662025": "160",
#                    "ANW3458911884641748847717292622025": "152",
#                    "APF5880998101801749455460325692025": "214",
#                    "AQU51953824073717484194111115282025": "190",
#                    "ASJ6220567876351749022864270642025": "183",
#                    "ATI2931037342771749455434963692025": "215",
#                    "ATO7840530765421748847498087622025": "165",
#                    "AUF8172309486891749021649962642025": "224",
#                    "AUX1823173115431749024421547642025": "156",
#                    "AYG16857793324417496271433326112025": "155",
#                    "AYJ37522687349917476378113375192025": "161",
#                    "BBD6435592255171749022900066642025": "184",
#                    "BCN4819847579361749455413872692025": "224",
#                    "BDT32843093613617484155933585282025": "222",
#                    "BDU5863966404271749455449375692025": "209",
#                    "BEG89982666272317478121718285212025": "209",
#                    "BEV88218588768817478134634625212025": "161",
#                    "BHN2575613242031749022804993642025": "198",
#                    "BKF9354522055301749195216600662025": "217",
#                    "BLL82517506170317478155644835212025": "182",
#                    "BOE5363648209021749020014791642025": "208",
#                    "BPL29621415380417478118399985212025": "224",
#                    "BPS8124300621041749193872592662025": "169",
#                    "BQG98438296849317484194229205282025": "191",
#                    "BUH85996003863917476412340825192025": "200",
#                    "BUT57993631852517485906574935302025": "207",
#                    "BUX42706939031017496252863236112025": "219",
#                    "BVM80819751031817484153920845282025": "204",
#                    "BXW33865938421517484152724165282025": "206",
#                    "BYZ7535125008501749455522329692025": "219",
#                    "CEA6946557649261749195227817662025": "211",
#                    "CEP4372202662901749024434639642025": "153",
#                    "CFP5453680501431749195280413662025": "215",
#                    "CHW38976571796417484192872055282025": "189",
#                    "CIS41429833215917496250582676112025": "216",
#                    "CJK56134644364517478117711165212025": "213",
#                    "CKL26894327256317476411678885192025": "177",
#                    "CLT4854018505971749193103449662025": "170",
#                    "CMI1212894850311749022840201642025": "189",
#                    "CMX82949275736017496270777736112025": "174",
#                    "CMY40962383263617484179602265282025": "151",
#                    "CMZ37227614754817478134530185212025": "154",
#                    "COX89218394584417496270667456112025": "158",
#                    "CPW80316338922917476410324355192025": "192",
#                    "CQL34476640988017476378331395192025": "159",
#                    "CQO6485787410621749195271344662025": "205",
#                    "CTD2971344580561749022931867642025": "195",
#                    "CVH2026397083651749196374545662025": "184",
#                    "CXB86269966541217496282965406112025": "186",
#                    "CXP1336718036371749454439595692025": "177",
#                    "CXT80956571292017485912169535302025": "203",
#                    "CYQ43447736959417478156301345212025": "179",
#                    "CYV2392268318461748850122629622025": "193",
#                    "CZP73696247061017484158859725282025": "223",
#                    "CZS29552407808617496284597246112025": "182",
#                    "DAQ15069043378017478122941305212025": "216",
#                    "DBQ5499612102491749196401288662025": "196",
#                    "DFP5688619861721749455427432692025": "203",
#                    "DFZ7743785975861749022820860642025": "179",
#                    "DGU5497506094261749196417350662025": "178",
#                    "DKD82329558341017496254885966112025": "204",
#                    "DKF11761060302317484179000005282025": "172",
#                    "DKR69961268491317484194273935282025": "201",
#                    "DLF3463182569431749195271871662025": "219",
#                    "DLT1602610875231749452455312692025": "156",
#                    "DQN2049096803531749193045901662025": "157",
#                    "DQW44808748661317485879303655302025": "183",
#                    "DTY67353804803217485920819925302025": "165",
#                    "DTZ7016601619071749020230969642025": "210",
#                    "DUJ69342601569517476308446865192025": "1",
#                    "DWZ3444472151511749023687543642025": "196",
#                    "DYU9142795055981749454394617692025": "187",
#                    "DZM93886065232517484158484605282025": "221",
#                    "ECP91843681021317496249945436112025": "223",
#                    "ECX76211263466217476377946615192025": "170",
#                    "EDL3805242346541749024575591642025": "169",
#                    "EGU1134341212631749196494963662025": "178",
#                    "EKU11893199230117496252731956112025": "202",
#                    "ELL91929294502317484194031915282025": "191",
#                    "ELM88362040935917485884408305302025": "195",
#                    "ENF27431930712917496283431826112025": "185",
#                    "ENQ5498964660351749452454415692025": "162",
#                    "ESO26571601099217485881494775302025": "193",
#                    "ETG7483973096491748850043100622025": "200",
#                    "EUE26659074356117496284533566112025": "178",
#                    "EWM4684735280251749196364082662025": "186",
#                    "EYS54585282895917496282815176112025": "184",
#                    "FAL77344149655617478136746985212025": "171",
#                    "FBG7500434064331748847697958622025": "151",
#                    "FCB66076015020317478117937275212025": "220",
#                    "FCR9478747666701748847382635622025": "159",
#                    "FDZ5508736367961749454432899692025": "181",
#                    "FEB4343973468451749454373855692025": "179",
#                    "FFI12348063160717478136816995212025": "163",
#                    "FND3924464941221749195250029662025": "206",
#                    "FPB5435421454561749024401875642025": "173",
#                    "FPW1912387362551749195656125662025": "211",
#                    "FPY28272197371717476376618755192025": "151",
#                    "FRD60715280340917485884205645302025": "184",
#                    "FSC61832273066817485881650245302025": "196",
#                    "FST2110753034301749022923805642025": "199",
#                    "FTB31118163097617484153584205282025": "205",
#                    "FTH3244555808821749020798575642025": "220",
#                    "FUJ98555068291817484170588255282025": "209",
#                    "FUX14957456438717478117777245212025": "210",
#                    "FZM38273687007517496283268136112025": "180",
#                    "GAA7184937302521749454476322692025": "178",
#                    "GAK3829603285811749020468505642025": "214",
#                    "GAM5849588374071749024424068642025": "175",
#                    "GHB91819536831617478155601435212025": "186",
#                    "GHE3531841156341749021715469642025": "217",
#                    "GHI25747535210917484180557055282025": "168",
#                    "GHU9393844941071749195242969662025": "210",
#                    "GLD8326233229251749024547021642025": "162",
#                    "GLT3546185042821749195223893662025": "203",
#                    "GMB4243498709471748850150543622025": "192",
#                    "GMT2473234574841749455893044692025": "212",
#                    "GNE86646635453517496271320296112025": "159",
#                    "GRA3212669377411749195312183662025": "222",
#                    "GTT14482228926417476411558765192025": "189",
#                    "GTY8967353763891749196389138662025": "192",
#                    "GVB7908770616071749454415698692025": "194",
#                    "GWR65519203702617484195421425282025": "184",
#                    "GXU13607600796617478135515435212025": "167",
#                    "GXZ2420973764121749455454097692025": "211",
#                    "GYX9283492748781749454415361692025": "200",
#                    "GZY79518050807017496283206366112025": "195",
#                    "HCF29680952450017484179207605282025": "175",
#                    "HCV7437617044141748850123232622025": "181",
#                    "HFA13593634117517476378269225192025": "173",
#                    "HFN35744703616717478136568755212025": "174",
#                    "HFQ5903115899441748847567376622025": "169",
#                    "HGC4684055427341748847313730622025": "168",
#                    "HHW18285963479817485919503865302025": "157",
#                    "HOI96853482775117485907436705302025": "213",
#                    "HOL71281032821417478135227875212025": "168",
#                    "HQF3050362262311749193547683662025": "174",
#                    "HSN71451153032317485919706245302025": "173",
#                    "HTZ12060570640117484189812015282025": "151",
#                    "HTZ6827195346361749455493966692025": "212",
#                    "HUU7199020979151748850165193622025": "198",
#                    "HWI13546454742617496283222146112025": "193",
#                    "HYG47172882235117478111273405212025": "218",
#                    "HZX9008513344601749023049916642025": "191",
#                    "IAU33277099434217496283310086112025": "196",
#                    "ICW35594717504217484194801125282025": "185",
#                    "ICY56540794418517476378397025192025": "169",
#                    "IEI61731017941417478114814135212025": "217",
#                    "IGX3998968396431749454493697692025": "184",
#                    "IJK9624917433401749020455640642025": "217",
#                    "ILF5695614932961749193136668662025": "158",
#                    "ILI49134200894917485907429165302025": "205",
#                    "ILX7592708050721749022995587642025": "181",
#                    "IMY1369501823221748850010755622025": "187",
#                    "IOI9814756332661749021021304642025": "207",
#                    "IQH5173303006041749455726701692025": "213",
#                    "IQT86986881849117485908710205302025": "210",
#                    "IQU87909217199417485915185655302025": "221",
#                    "ISE21039096173217496270657216112025": "164",
#                    "ITJ81595029707217478155999195212025": "191",
#                    "IUF4964483314201749454405949692025": "189",
#                    "IUQ44898165825117476412160185192025": "182",
#                    "IWV5612424372701749020686718642025": "223",
#                    "IXJ6462974135181749454448993692025": "197",
#                    "IYN71570550136017496270255166112025": "175",
#                    "IZN31234433868417478117567765212025": "203",
#                    "JBP28055498620617485920136635302025": "155",
#                    "JCD1932979542511749193387113662025": "153",
#                    "JDB6104245929091749020267458642025": "206",
#                    "JEB81269541978617485921758595302025": "167",
#                    "JED86333888881117496252452806112025": "213",
#                    "JER9697964988151749193319449662025": "152",
#                    "JFZ75619926536617484179698885282025": "167",
#                    "JGQ5536099811281749452426059692025": "168",
#                    "JGS2553169853871749193168330662025": "164",
#                    "JIE59562986521717484152322815282025": "215",
#                    "JIN5795089954831749022905042642025": "182",
#                    "JJF7858956517951748847369557622025": "158",
#                    "JLW8117621611561749452340870692025": "152",
#                    "JLW92279287284617484180158785282025": "154",
#                    "JPA65975659720717484154174095282025": "216",
#                    "JQX6522362679311748847737344622025": "154",
#                    "JSJ2508389761251749021817347642025": "216",
#                    "JUK19769901163117478136072855212025": "159",
#                    "JUW59338787271117485883301425302025": "181",
#                    "JVI6150349196431748847263861622025": "160",
#                    "JYK26631858182417478136938575212025": "170",
#                    "JYK9470977141271749024505165642025": "160",
#                    "KAA3193695343321748849986164622025": "195",
#                    "KAQ73632638286317484195554145282025": "182",
#                    "KDL48845223789117476410935275192025": "196",
#                    "KDU11716723282417485906618505302025": "218",
#                    "KEZ9713200402221749195205837662025": "216",
#                    "KFI9173492459251749193033209662025": "175",
#                    "KFZ3115209964031749455836450692025": "205",
#                    "KGG83501931293817478155603805212025": "200",
#                    "KHH28763459597117478156494545212025": "194",
#                    "KHT37032808329317484159442475282025": "224",
#                    "KIK2291504233101749195265642662025": "209",
#                    "KIO57223880418017485907965485302025": "208",
#                    "KIQ8613468421611749454454223692025": "196",
#                    "KKI9365434289921749455444047692025": "222",
#                    "KMO3733965759931749455413989692025": "207",
#                    "KNU3135953307941749196406346662025": "195",
#                    "KRX3155716686621749021047219642025": "202",
#                    "KSH5399817788421749024574097642025": "167",
#                    "KTG6437427289891749195261413662025": "220",
#                    "KTV60392887473817485919218585302025": "152",
#                    "KUJ80391314269417478136255085212025": "155",
#                    "KUL7912377409731749021713623642025": "203",
#                    "KXK3627892744751748848846884622025": "163",
#                    "KXP15608511956617478156755155212025": "197",
#                    "KXR37887510047517496285400616112025": "201",
#                    "KXU5379594202481749021074348642025": "216",
#                    "KYD96464017317817478157050445212025": "183",
#                    "LCU39845838584817496250547806112025": "209",
#                    "LDT24910791698717485896232225302025": "178",
#                    "LFJ2442785401361749022915616642025": "187",
#                    "LGK78623260024117484180436865282025": "171",
#                    "LGP35011225391817496271597656112025": "168",
#                    "LIL47925654182617485908536315302025": "203",
#                    "LIW2486213278081749193246316662025": "163",
#                    "LJE17280986558917485919925055302025": "170",
#                    "LJP9302704728091749454381774692025": "192",
#                    "LLV2916689304021749020412417642025": "213",
#                    "LMC90711908873317485880299015302025": "192",
#                    "LNO40128976505517485914952335302025": "208",
#                    "LOA5745326257861749452401735692025": "153",
#                    "LQK1016626263321749195208575662025": "208",
#                    "LRJ49143005332817485920003005302025": "175",
#                    "LSK1789226633021749024496277642025": "151",
#                    "LTK3015660641931749022192315642025": "205",
#                    "LTU44724917262517485908476745302025": "223",
#                    "LVA50771989199417484155776945282025": "208",
#                    "LVS9144487430901749021643848642025": "208",
#                    "LYX65615991846117496270673996112025": "165",
#                    "LZP7914051894161749452348901692025": "154",
#                    "MAQ33550440093817476411850345192025": "180",
#                    "MBO88058776803317485886262465302025": "197",
#                    "MBX4951147888151749196365934662025": "179",
#                    "MCR11679714450717496271016016112025": "162",
#                    "MID9770545977861749193125250662025": "159",
#                    "MIG83288914355717476410940365192025": "185",
#                    "MIL42181934086117496282397566112025": "177",
#                    "MJM76750115238017485879288475302025": "191",
#                    "MKG42155107642017485919426355302025": "151",
#                    "MLS5641363279071749022904370642025": "193",
#                    "MML84215705403617478135807745212025": "162",
#                    "MMT87645271228717485906971135302025": "215",
#                    "MOJ3441344427791748850220104622025": "194",
#                    "MPI85468194084517484194121715282025": "186",
#                    "MQF6448877474391749452406999692025": "166",
#                    "MTJ66828295309817496284130166112025": "181",
#                    "MUC24871038063517484158409705282025": "220",
#                    "MUH72591305117617484155782135282025": "209",
#                    "MUQ98577881553117478114523585212025": "204",
#                    "MUV33647633654517485885991955302025": "177",
#                    "MWX7617844947501748850032922622025": "189",
#                    "MYC74698875159817485921328565302025": "225",
#                    "MYG11311190403617485908689975302025": "224",
#                    "MZB14706317479017496270993656112025": "171",
#                    "MZG2292602001611749196371209662025": "199",
#                    "MZO12806086387817485919364295302025": "158",
#                    "NAA81312113762717496272093206112025": "154",
#                    "NCM74921561285317496282583696112025": "197",
#                    "NEF42029055850317496283552826112025": "194",
#                    "NES6170456483561749454404719692025": "185",
#                    "NGN19501531037017496283588336112025": "181",
#                    "NIU9680407953041749022805482642025": "192",
#                    "NJG37551151593317484153870295282025": "218",
#                    "NJG9139144962521749452119871692025": "161",
#                    "NJJ36447104155817478134307215212025": "166",
#                    "NKJ8313977745511749196236109662025": "180",
#                    "NKL18081303934817496254433156112025": "218",
#                    "NLQ68669833305117484180897715282025": "170",
#                    "NMK55368239735617484194435995282025": "192",
#                    "NMO28653325261817484180944895282025": "169",
#                    "NOJ3512597354781749020996280642025": "205",
#                    "NOS1130741989831749021638426642025": "212",
#                    "NPE3337834813961749455474162692025": "205",
#                    "NQG55224446163217484179412485282025": "166",
#                    "NRG34018179877617478155879325212025": "185",
#                    "NSL8004866417141749196496081662025": "183",
#                    "NSN3815457173041749455468473692025": "217",
#                    "NUW2793919261401749021603251642025": "213",
#                    "NYC1319578795321749024480531642025": "159",
#                    "NYD66059072817217485919906245302025": "171",
#                    "NZA2848632153661748850162805622025": "185",
#                    "NZA50266819760017484180972755282025": "158",
#                    "NZL4613523597271749022967467642025": "178",
#                    "OAW13416112854017484179506655282025": "173",
#                    "OEB5721971699011749024566117642025": "163",
#                    "OEX18601261671417478156549935212025": "199",
#                    "OFC40497766613517478116826835212025": "205",
#                    "OGT55925886828517485908356465302025": "220",
#                    "OIQ9312244246901749193112503662025": "156",
#                    "OJQ9158772425511749021687786642025": "221",
#                    "OKJ19293346021517485906583645302025": "216",
#                    "ONH47323010310617484193781465282025": "195",
#                    "ONR60305128404717496282741476112025": "183",
#                    "OPF5475442915231749454406003692025": "193",
#                    "OPP1329853322701749023030275642025": "194",
#                    "OQM7844869640471749193266372662025": "151",
#                    "OTQ9387934411481749022877228642025": "186",
#                    "OUH16469425976917484185482025282025": "167",
#                    "OZA78577872161617476412259575192025": "188",
#                    "OZI43019491006017478137272345212025": "173",
#                    "PBI7151999899871749455434718692025": "223",
#                    "PBR4469528119241749021682242642025": "223",
#                    "PCN2805540268461749455394947692025": "208",
#                    "PEC74135239975417478117346355212025": "208",
#                    "PGD6034545249611748850008842622025": "177",
#                    "PGE2680253906741749195223579662025": "207",
#                    "PGV2055312915751749195207775662025": "202",
#                    "PHX69955418307317478155774745212025": "189",
#                    "PJA98561581980917478134636155212025": "165",
#                    "PJX62860787878917485908293285302025": "209",
#                    "PKC6296503917241749021849862642025": "208",
#                    "PKE4034699695111749020696400642025": "224",
#                    "PLO4471030095821748849110454622025": "154",
#                    "PMW49224790241417484153295605282025": "212",
#                    "PMX11945272195217478156528965212025": "201",
#                    "PNT61726242986817478141058395212025": "160",
#                    "PNX94853030970817496251600796112025": "207",
#                    "PQA10364151503517496284584116112025": "198",
#                    "PQA2900849633661749021723131642025": "204",
#                    "PRE7267781581511749023031688642025": "185",
#                    "PRO62315252064317484194050455282025": "181",
#                    "PRX30612099178617484193534275282025": "178",
#                    "PSV9412228316781749454445659692025": "195",
#                    "PUE8938136180701749020021955642025": "211",
#                    "PWE20844503265617478135463485212025": "157",
#                    "PWN5683201485881749452448392692025": "157",
#                    "PWU37418213087417485907346215302025": "219",
#                    "PXD2772609513061749021562062642025": "214",
#                    "PXH5293628260871749022833925642025": "201",
#                    "PXS5515387265411749455670781692025": "210",
#                    "PYJ10394986287817485914596715302025": "219",
#                    "QAK99373554555517496272149776112025": "166",
#                    "QCA47506876930017484154826165282025": "213",
#                    "QCL85374090442217496253928166112025": "203",
#                    "QCN8096307538681749020637244642025": "221",
#                    "QDQ87330552516917476411143475192025": "201",
#                    "QDX16794273816717485929515255302025": "174",
#                    "QER7032251979281748850018452622025": "191",
#                    "QER9338307802911749454495553692025": "182",
#                    "QHJ5287329779811749454462701692025": "199",
#                    "QJI1147273045751749454568995692025": "178",
#                    "QKN12753115766517485920247755302025": "160",
#                    "QLA4083598270491748850168361622025": "197",
#                    "QLR4949040915511748850072730622025": "196",
#                    "QNB57983566505417478156076205212025": "178",
#                    "QRD89588591131717485912948735302025": "222",
#                    "QRK64700083980617484195945775282025": "194",
#                    "QUB30256493594917476379992925192025": "175",
#                    "QUJ97019003092517484180733075282025": "165",
#                    "QYT11765460181017478118657285212025": "206",
#                    "QYU72980993731417478156757745212025": "193",
#                    "QZI7341797502391748849183763622025": "169",
#                    "RAL37447895932317478120763195212025": "223",
#                    "RBG33590877495717478113543125212025": "212",
#                    "RCF78235285703417484159325535282025": "202",
#                    "RCZ28991222872617496270585306112025": "173",
#                    "RDJ6021785806921749196382015662025": "197",
#                    "RDW6524091240811749196327632662025": "189",
#                    "REM6383000528161748850014244622025": "179",
#                    "RFN6528995847301749193134879662025": "155",
#                    "RFW12453094999917478156702425212025": "184",
#                    "RKL37451423560217496271145116112025": "163",
#                    "RKQ2084471947751749020738651642025": "222",
#                    "RKV5142043992531749024528375642025": "155",
#                    "RKX56492720072517485908045745302025": "214",
#                    "RLF21748694026517496282633036112025": "200",
#                    "RMD8053919937251749196371132662025": "190",
#                    "RMX22442431760517484180498885282025": "161",
#                    "ROU16164886609617484193008285282025": "188",
#                    "RPE10240317680217484179233905282025": "152",
#                    "RQT6646288092621748847726390622025": "153",
#                    "RSZ7297759900241749024530607642025": "165",
#                    "RTH4524791995691749024516937642025": "164",
#                    "RTK3909812600641749193247933662025": "167",
#                    "RUN33447979806617484180354775282025": "164",
#                    "RVR52528449258517496252682736112025": "212",
#                    "RVU29433249757417485885468825302025": "182",
#                    "RVU30175738808217485919799725302025": "153",
#                    "RXB79712170172617496272305196112025": "151",
#                    "RXN55469736953817484179889655282025": "153",
#                    "SDM21992777718717485885743265302025": "179",
#                    "SEL1079236072511748850233526622025": "178",
#                    "SFW63872312761817485883004515302025": "187",
#                    "SGS26791731120617484152584895282025": "214",
#                    "SHL4187774876671749195301483662025": "223",
#                    "SHO46598687967817476318608045192025": "1",
#                    "SHY69216516742917484180181385282025": "155",
#                    "SIG7128779332501749454414327692025": "190",
#                    "SKR74433170373517484154365975282025": "217",
#                    "SLX9339425580601749196354284662025": "182",
#                    "SNG9317598830691749196309380662025": "200",
#                    "SPA2426872539451749455390460692025": "202",
#                    "SSQ73464475498217485883659265302025": "199",
#                    "SUS86771887174617478117477975212025": "221",
#                    "SUV20673791994417478155806265212025": "181",
#                    "SYL6540115129721748850027742622025": "180",
#                    "SZD8429814001851749022887575642025": "200",
#                    "SZE8829055961111748847460890622025": "175",
#                    "SZK4281979250941748847052047622025": "170",
#                    "SZU67127916319017496271965026112025": "169",
#                    "TAC2354815932671749020172720642025": "204",
#                    "TEF9787885959541749195224612662025": "224",
#                    "TEQ86547097891817496270459296112025": "160",
#                    "TFB86549833471817484157695185282025": "163",
#                    "TGF68498440779917484194171885282025": "179",
#                    "TGG68603260357217478156634665212025": "192",
#                    "TIN11727622483817485906648135302025": "202",
#                    "TJB21268846676217478136997735212025": "175",
#                    "TKZ22983072856417496252149796112025": "211",
#                    "TOQ9637267310201749195320330662025": "221",
#                    "TPG23808150139417496283446026112025": "179",
#                    "TQK91935408928117478135226345212025": "152",
#                    "TRT16763369894117478113785085212025": "214",
#                    "TSC61277663104717485921309095302025": "166",
#                    "TTE5533876908791749455431204692025": "216",
#                    "TTN83894436857217496271778866112025": "152",
#                    "TUQ3814064103741749024511541642025": "152",
#                    "TYE37491744391217484179831825282025": "157",
#                    "TYG7846191596921749195287219662025": "212",
#                    "UAO39468705532217485920701635302025": "162",
#                    "UCK86392294316417496283001606112025": "199",
#                    "UCX25834454182417484194766995282025": "196",
#                    "UEI44011485959717476411525915192025": "179",
#                    "UGC58708513463417485908780545302025": "221",
#                    "UGC7056757232861749024423057642025": "158",
#                    "UGH2084537757561749195246195662025": "204",
#                    "UGJ2272487266541748847753206622025": "156",
#                    "UGY43692854402717496271331586112025": "170",
#                    "UIO85815363158617485913337765302025": "216",
#                    "UKI27728168443817485920370405302025": "164",
#                    "UKO59604222571717484153732125282025": "219",
#                    "UKX4193262944731749022931339642025": "188",
#                    "UME2001609760261749454531249692025": "182",
#                    "UOU54725906978917478156343595212025": "190",
#                    "UPL3961779744031748849998018622025": "182",
#                    "UQS2321893687521749452288659692025": "170",
#                    "URH35627041526217496250812856112025": "205",
#                    "USF98036991398017496282951976112025": "187",
#                    "USL7344708406161749455431656692025": "206",
#                    "UUU2426780761521749195234066662025": "218",
#                    "UVE47018850994617485920460495302025": "156",
#                    "UVQ88886452145717476411172705192025": "194",
#                    "VAY7952358559541748847488164622025": "163",
#                    "VCN9797353926921749024399075642025": "157",
#                    "VEY9331087803041749195299147662025": "214",
#                    "VFH8517150954291749196396283662025": "188",
#                    "VJU22424132803717485907474155302025": "212",
#                    "VKE73800622347917485879292985302025": "185",
#                    "VKE8758774007571749195303592662025": "213",
#                    "VKJ56390246606217484155333525282025": "207",
#                    "VKM1844130947101749196396170662025": "177",
#                    "VMW4341539719571749024514131642025": "170",
#                    "VNY50502582620717485885790475302025": "194",
#                    "VOD9591380678651749024544502642025": "166",
#                    "VPP59262368006217496282982006112025": "192",
#                    "VQL43821621032117484192933965282025": "178",
#                    "VRV7593282504881749193245805662025": "162",
#                    "VTO3224156429341749024472190642025": "154",
#                    "VUB5555565517131749196432955662025": "194",
#                    "VUJ16305310326717485882533055302025": "188",
#                    "VVO5716608666181748847620354622025": "167",
#                    "VVO8552317746651749455607135692025": "218",
#                    "VWB1726094820001748850016570622025": "184",
#                    "VWF2591577959261748847080244622025": "171",
#                    "VXV98076084716717478135820615212025": "160",
#                    "VYE35336136799817476375620755192025": "168",
#                    "VZN51298783674217484193916155282025": "187",
#                    "WES41921974469317485920012205302025": "159",
#                    "WGO4748396074141748847612030622025": "155",
#                    "WHS34130163792817484178982165282025": "159",
#                    "WHV1266886129151749196393849662025": "193",
#                    "WJN63446613229017478118415555212025": "211",
#                    "WJS57940694447017478121522685212025": "207",
#                    "WKO79186777496317476378081085192025": "172",
#                    "WKV3211564238881748847562938622025": "166",
#                    "WMT57659782501617484179803565282025": "160",
#                    "WSE38049791348617496250324516112025": "224",
#                    "WSW4646928713091749021667671642025": "209",
#                    "WUS97191836384017484173747915282025": "212",
#                    "WVB6803177193201749452317848692025": "172",
#                    "WXP5940474200891749455479152692025": "213",
#                    "WYF86914908489317485906916915302025": "211",
#                    "WZD48437648526817496255054096112025": "206",
#                    "WZL5870539946991749454394052692025": "180",
#                    "XED85825052058417496253095426112025": "217",
#                    "XEI52470680428517478163097455212025": "189",
#                    "XEO28887396918917478156814475212025": "198",
#                    "XFV6229339854601749021620181642025": "218",
#                    "XGF6730011766061749193141060662025": "172",
#                    "XHX6297992991611748847480258622025": "162",
#                    "XIK3984943175751749193227550662025": "168",
#                    "XKL28695250178417476379498475192025": "174",
#                    "XKP69847963305017485885412525302025": "180",
#                    "XLH77540927916217476411170945192025": "186",
#                    "XML52492407771017478156328785212025": "196",
#                    "XMX75952971805617496282838246112025": "190",
#                    "XNC5951201389281749452360002692025": "174",
#                    "XNQ3905122463651748850073465622025": "201",
#                    "XNS62529383813317478155794035212025": "188",
#                    "XNU61170794578817484180165035282025": "174",
#                    "XNW8464126167121749452310837692025": "151",
#                    "XPI34225176190617478135034085212025": "169",
#                    "XPT2531177519061749020975988642025": "215",
#                    "XQB26599658881617476411204645192025": "199",
#                    "XSU9309292767731749453784480692025": "159",
#                    "YBA9960000209021749021670758642025": "205",
#                    "YBI6451685389671749196329588662025": "185",
#                    "YDX43656575920617496272165666112025": "167",
#                    "YFY5011151417001749193083456662025": "171",
#                    "YIB18419722767117485908058995302025": "206",
#                    "YOP7634275239211749193361586662025": "154",
#                    "YOQ67719183856717496271733726112025": "156",
#                    "YQE3461163948731749024641924642025": "168",
#                    "YUA8349556461721749452449775692025": "167",
#                    "YUR70836477674817485879335995302025": "186",
#                    "YYV27144167584017496250765286112025": "215",
#                    "YZE88923351848417496270942516112025": "165",
#                    "YZS96097609827617496282722266112025": "189",
#                    "ZAC52202721226117484155671415282025": "211",
#                    "ZAP97516618104117484192974005282025": "180",
#                    "ZBB2368033936321749455480149692025": "204",
#                    "ZBJ42608369169617496249541966112025": "220",
#                    "ZEI88189702993217478134139565212025": "153",
#                    "ZGE6641316565591749020208390642025": "203",
#                    "ZHJ26094984824917485881700805302025": "200",
#                    "ZHV48248959766917478156948215212025": "197",
#                    "ZII25681360056617485920522745302025": "168",
#                    "ZIN17085806547917478121327575212025": "222",
#                    "ZIP59577876777617476376604205192025": "171",
#                    "ZJH71657434483717478134336395212025": "156",
#                    "ZLQ56491195546517478126856435212025": "214",
#                    "ZLT70706628196417485908205765302025": "217",
#                    "ZLZ4515798975271749020251848642025": "209",
#                    "ZOL52597924054517485907142435302025": "204",
#                    "ZOM5046903491601749022989706642025": "196",
#                    "ZPN1342952073791749196424464662025": "201",
#                    "ZPX9005965327731749193211675662025": "165",
#                    "ZTY4789784275601749454448938692025": "186",
#                    "ZWT78096372496017484180143735282025": "162",
#                    "ZWU2332400682681748847042651622025": "172",
#                    "ZYK92489163737217478134252025212025": "164",
#                    "QNH66810377396417496270807826112025": "161",
#                    "RHF9776305947981749021794017642025": "218",
#                    "TSR1647064472351748850038241622025": "199",
#                    "UVT1936854193581749022935335642025": "180",
#                    "VBO65412176323017485880906575302025": "201",
#                    "VOT4372363462831749022811990642025": "177",
#                    "VRF16527753060717478164191705212025": "178",
#                    "WJW3787442081651748847644910622025": "157",
#                    "WRC43674233253617496250878236112025": "208",
#                    "WSB8315790574491749452478325692025": "169",
#                    "XIA7466900567371749452474620692025": "163",
#                    "XOY7994148270991749455696102692025": "206",
#                    "XWK20950159753617484179599925282025": "156",
#                    "YQY1076343372741749022816149642025": "190",
#                    "YUV62314785782317476404390705192025": "163",
#                    "ZZO9975957222021749455484242692025": "220"}
#
# df_filtered['userId'] = df_filtered['sessionId'].map(session_to_user)
#
# scores = {
#     "221": "86",
#     "183": "15",
#     "197": "4",
#     "198": "24",
#     "178": "93",
#     "215": "104",
#     "214": "79",
#     "190": "48",
#     "224": "65",
#     "184": "13",
#     "222": "40",
#     "209": "9",
#     "217": "11",
#     "182": "75",
#     "208": "90",
#     "191": "20",
#     "200": "24",
#     "207": "8",
#     "219": "30",
#     "204": "91",
#     "206": "4",
#     "211": "14",
#     "189": "7",
#     "216": "33",
#     "213": "19",
#     "177": "16",
#     "192": "113",
#     "205": "99",
#     "195": "1",
#     "186": "13",
#     "203": "8",
#     "179": "11",
#     "193": "1",
#     "223": "103",
#     "196": "8",
#     "201": "19",
#     "210": "15",
#     "1": "69",
#     "187": "36",
#     "202": "17",
#     "185": "28",
#     "220": "10",
#     "181": "11",
#     "199": "11",
#     "180": "67",
#     "212": "35",
#     "194": "7",
#     "218": "69",
#     "188": "95",
#     "171": "0",
#     "157": "40",
#     "175": "0",
#     "166": "34",
#     "153": "10",
#     "172": "0",
#     "152": "36",
#     "158": "45",
#     "169": "2",
#     "160": "3",
#     "164": "46",
#     "165": "0",
#     "156": "0",
#     "155": "43",
#     "161": "0",
#     "174": "0",
#     "163": "0",
#     "151": "9",
#     "170": "1",
#     "154": "8",
#     "159": "0",
#     "168": "0",
#     "162": "51",
#     "173": "0",
#     "167": "22",
# }
# df_filtered['score'] = df_filtered['userId'].map(scores)

In [12]:
# df_filtered.dropna(subset=['score'], inplace=True)
#
# df_filtered = df_filtered[df_filtered['userId'] != '1']

In [13]:
#df_filtered.to_csv('data_filtered.csv', index=False)

In [14]:
import pandas as pd

dataset_path = 'data_filtered.csv'
df = pd.read_csv(dataset_path)
df.isna().mean()

sessionId                0.000000
sceneId                  0.000000
duration_ms_mean         0.000000
duration_ms_max          0.000000
duration_ms_min          0.000000
duration_ms_std          0.026076
total_distance_mean      0.000000
total_distance_max       0.000000
total_distance_min       0.000000
total_distance_std       0.026076
avg_velocity_mean        0.000000
avg_velocity_max         0.000000
avg_velocity_min         0.000000
avg_velocity_std         0.026076
max_velocity_mean        0.000000
max_velocity_max         0.000000
max_velocity_min         0.000000
max_velocity_std         0.026076
avg_acceleration_mean    0.000000
avg_acceleration_max     0.000000
avg_acceleration_min     0.000000
avg_acceleration_std     0.026076
max_acceleration_mean    0.000000
max_acceleration_max     0.000000
max_acceleration_min     0.000000
max_acceleration_std     0.026076
area_real_mean           0.000000
area_real_max            0.000000
area_real_min            0.000000
area_real_std 

In [15]:
import numpy as np
from sklearn.impute import SimpleImputer

numeric_cols = df.select_dtypes(include=[np.number]).columns
imputer = SimpleImputer(strategy='median')
df[numeric_cols] = imputer.fit_transform(df[numeric_cols])
df['ld'] = (df['score'] > 65).astype(int)
df.drop(columns=['score'], inplace=True)

In [16]:
import scipy.stats as stats

features = df.columns.drop(['userId', 'ld', 'sessionId', 'sceneId'])

p_values = {}

for col in features:
    group0 = df[df['ld'] == 0][col]
    group1 = df[df['ld'] == 1][col]

    stat, p = stats.ttest_ind(group0, group1, nan_policy='omit')
    p_values[col] = p

t_results = pd.DataFrame.from_dict(p_values, orient='index', columns=['p_value'])
t_results = t_results.sort_values(by='p_value')

In [17]:
t_results[t_results['p_value'] < 0.05]

,p_value
duration_ms_std,2.411996e-40
duration_ms_max,8.871093e-29
num_points_sum,1.120748e-21
total_distance_std,1.199388e-18
total_distance_max,1.151895e-16
total_distance_min,2.394954e-16
duration_ms_mean,1.758054e-14
duration_ms_min,7.322656e-12
max_velocity_max,4.566748e-11
num_points_mean,2.993099e-09
